In [ ]:
from IPython.core.display import HTML
with open('../style.css') as f:
    css = f.read()
HTML(css)

# Alpha-Beta Pruning with Intelligent Memoization

In order to have some variation in our game, we use random numbers to choose between optimal moves.
In order to have reproducible results, we use a *seed* for the random number generator.

In [ ]:
import random
random.seed(42)

The global variable `gCache` is used as a cache for the function `evaluate` defined later.  Instead of just storing the values for a given `State`, the cache stores pairs of the form 
* `('=', v)`, 
* `('≤', v)`, or
* `('≥', v)`.

The first component of these pairs is a *flag* that specifies whether the stored value `v` is exact or whether it only is a lower or upper bound.  Concretely, provided `gCache[State]` is defined and `evaluate(State)` computes the *value* of a given `State`, the following invariants are satisfied:
* $\texttt{gCache[State]} = (\texttt{'='}, v) \rightarrow \texttt{evaluate(State)} = v$.
* $\texttt{gCache[State]} = (\texttt{'≤'}, v) \rightarrow \texttt{evaluate(State)} \leq v$.
* $\texttt{gCache[State]} = (\texttt{'≥'}, v) \rightarrow \texttt{evaluate(State)} \geq v$.

In [ ]:
gCache = {}

The function `evaluate` takes four arguments:
- `State` is the current state of the game,
- `f`     is either the function `maxValue` or the function `minValue`.

   These functions are defined later. If in `State` it is the first player's turn, then `f` is equal to `maxValue`, else `f` is equal to `minValue`.
- `alpha` is a lower bound for the value of `State`,
- `beta`  is an upper bound for the value of `State`,

The function `evaluate` satisfies the following specification:
- $\alpha \leq \texttt{value}(s) \leq \beta \;\rightarrow\;\texttt{evaluate}(s, f, \alpha, \beta) = \texttt{value}(s)$
- $\texttt{value}(s) < \alpha \;\rightarrow\; \texttt{evaluate}(s, f, \alpha, \beta) \leq \alpha$
- $\beta < \texttt{value}(s) \;\rightarrow\; \beta \leq \texttt{evaluate}(s, f, \alpha, \beta)$

Here, the expression `value(s)` returns the *value* of the given state `s` if both players play their optimal game.  This value is an element from the set $\{-1, 0, 1\}$.  
- If the first player can force a win, the return value is `1`.
- If the first player can at best force a draw, the return value is `0`.
- If the second player can force a win, the return value is `-1`.

For reasons of efficiency, the function `evaluate` is *memoized* using the global variable `gCache`.
If `gCache[State]` is defined, then the computation of `evaluate(State, alpha, beta)`
proceeds according to the following case distinction:
1. If the stored value $v$ is exact, we can return this value:
   $$\texttt{gCache}[s] = (=, v) \rightarrow \texttt{evaluate}(s, f, \alpha, \beta) = v$$
2. If the stored value $v$ is an upper bound, then there are two cases.
   * If this upper bound $v$ is less or equal than $\alpha$, then we know that
     the true value of the state $s$ is less or equal than $\alpha$ and hence we can also      return the value $v$:
     $$\texttt{gCache}[s] = (\leq, v) \wedge v \leq \alpha \rightarrow 
     \texttt{evaluate}(s, f, \alpha, \beta) = v$$
   * Otherwise we can sharpen the upper bound $\beta$ by setting $\beta$ to be the minimum      of $\beta$ and $v$:
     $$\texttt{beta} = \texttt{min}(\texttt{beta}, v)$$
     If this leads to a reduction of $\beta$, then size of the interval 
     $[\alpha, \beta]$ is reduced and hence Alpha-Beta pruning will be able to remove 
     more nodes from the game tree, making the search more efficient.
3. If the stored value $v$ is a lower bound, there are again two cases.
   * If this lower bound is greater or equal than $\beta$, then we 
     know that the true value is bigger or equal than $\beta$ and hence we can 
     return the value $v$: 
     $$\texttt{gCache}[s] = (\geq, v) \wedge \beta \leq v \rightarrow 
     \texttt{evaluate}(s, f, \alpha, \beta) = v$$
   * Otherwise, we can sharpen the lower bound $\alpha$ by setting $\alpha$ to be the 
     maximum of $\alpha$ and $v$:
     $\texttt{alpha} = \texttt{max}(\texttt{alpha}, v)$
     If this leads to an increase of $\alpha$, then size of the interval 
     $[\alpha, \beta]$ is reduced and hence Alpha-Beta pruning will be able to 
     remove more nodes from the game tree, making the search more efficient.

In [ ]:
def evaluate(State, f, alpha=-1, beta=1):
    global gCache
    if State in gCache:
        flag, v = gCache[State]
        if flag == '=':
            return v
        if flag == '≤':
            if v <= alpha:
                return v
            else:
                beta = min(beta, v)
        if flag == '≥':
            if beta <= v:
                return v
            else:
                alpha = max(alpha, v)
    v = f(State, alpha, beta)
    store_cache(State, alpha, beta, v)
    return v

The function `store_cache` is called with four arguments:
* `State` is a state of the game,
* `alpha` is a number,
* `beta`  is a number, and
* `v`     is a number such that:
  $$\texttt{evaluate(State, f, alpha, beta)} = v$$
  
The function stores the `value` in the dictionary `gCache` under the key `State`.
It also stores an indicator that is either `'≤'`, `'='`, or `'≥'`.  The value that is stored 
satisfies the following conditions:
* If `gCache[State] = ('≤', v)`, then `value(State) ≤ v`. 
* If `gCache[State] = ('=', v)`, then `value(State) = v`. 
* If `gCache[State] = ('≥', v)`, then `value(State) ≥ v`. 

In [ ]:
def store_cache(State, alpha, beta, v):
    global gCache
    if   v <= alpha:
        gCache[State] = ('≤', v)
    elif v <  beta: # alpha < v
        gCache[State] = ('=', v)
    else: # beta <= v
        gCache[State] = ('≥', v)

The function `maxValue` is called with three arguments:
* `State` is a state of the game,
* `alpha` is a number, and
* `beta`  is a number.

The function `maxValue` is only called if in the given `State` it is the turn of the maximizing player.

The function `maxValue` satisfies the following specification:
- $\alpha \leq \texttt{value}(s) \leq \beta \;\rightarrow\;\texttt{maxValue}(s, \alpha, \beta) = \texttt{value}(s)$
- $\texttt{value}(s) \leq \alpha \;\rightarrow\; \texttt{maxValue}(s, \alpha, \beta) \leq \alpha$
- $\beta \leq \texttt{value}(s) \;\rightarrow\; \beta \leq \texttt{maxValue}(s, \alpha, \beta)$

In [ ]:
def maxValue(State, alpha, beta):
    if finished(State):
        return utility(State)
    for ns in next_states(State, gPlayers[0]):
        value = evaluate(ns, minValue, alpha, beta)
        if value >= beta:
            return value
        alpha = max(alpha, value)
    return alpha

The function `minValue` is called with three arguments:
* `State` is a state of the game,
* `alpha` is a number, and
* `beta`  is a number.

The function `minValue` is only called if in the given `State` it is the turn of the minimizing player.

The function `minValue` satisfies the following specification:
- $\alpha \leq \texttt{value}(s) \leq \beta \;\rightarrow\;\texttt{minValue}(s, \alpha, \beta) = \texttt{value}(s)$
- $\texttt{value}(s) \leq \alpha \;\rightarrow\; \texttt{minValue}(s, \alpha, \beta) \leq \alpha$
- $\beta \leq \texttt{value}(s) \;\rightarrow\; \beta \leq \texttt{minValue}(s, \alpha, \beta)$

In [ ]:
def minValue(State, alpha, beta):
    if finished(State):
        return utility(State)
    for ns in next_states(State, gPlayers[1]):
        value = evaluate(ns, maxValue, alpha, beta)
        if value <= alpha:
            return value
        beta = min(beta, value)
    return beta

In [ ]:
%%capture
%run 2-Tic-Tac-Toe-Bitboard.ipynb

$\alpha$-$\beta$ pruning with intelligent memoization takes 17 ms to compute the value of the state `gStart` for Tic-Tac-Toe. 
Hence, there is no big difference between intelligent memoization and naive memoization in this example.  However, if we implement
a non-trivial game like Connect Four, the situation changes.

In [ ]:
%%time
v = evaluate(gStart, maxValue, -1, 1)
v

We check how many different states are stored in the `Cache`.  Without alpha-beta pruning, we had to inspect $5478$ different states, but now there are only
$2474$ different states in the cache.

In [ ]:
len(gCache)

## Playing the Game

The function `best_move` takes two arguments:
- `State` is the current state of the game,
- `player` is a player.

The function `best_move` returns a pair of the form $(v, s)$ where $s$ is a state and $v$ is the value of this state.  The state $s$ is a state that is reached from `State` if `player` makes one of her optimal moves.  In order to have some variation in the game, the function randomly chooses any of the optimal moves.

In [ ]:
def best_move(State):
    NS        = next_states(State, gPlayers[0])
    bestValue = evaluate(State, maxValue, -1, 1)
    BestMoves = [s for s in NS if evaluate(s, minValue, -1, 1) == bestValue]
    BestState = random.choice(BestMoves)
    return bestValue, BestState

The next line is needed because we need the function `IPython.display.clear_output` to clear the output in a cell.

In [ ]:
import IPython.display 

The function `play_game` plays on the given `canvas`.  The game played is specified indirectly by specifying the following:
- `Start` is a global variable defining the start state of the game.
- `next_states` is a function such that $\texttt{next_states}(s, p)$ computes the set of all possible states that can be reached from state $s$ if player $p$ is next to move.
- `finished` is a function such that $\texttt{finished}(s)$ is true for a state $s$ if the game is over in state $s$.
- `utility` is a function such that $\texttt{utility}(s, p)$ returns either `-1`, `0`, or `1` in the *terminal state* $s$.  We have that
  - $\texttt{utility}(s, p)= -1$ iff the game is lost for player $p$ in state $s$, 
  - $\texttt{utility}(s, p)=  0$ iff the game is drawn, and 
  - $\texttt{utility}(s, p)=  1$ iff the game is won for player $p$ in state $s$.

In [ ]:
def play_game(canvas):
    State = gStart
    while (True):
        val, State = best_move(State);
        draw(State, canvas, f'For me, the game has the value {val}.')
        if finished(State):
            final_msg(State)
            break
        IPython.display.clear_output(wait=True)
        State = get_move(State)
        draw(State, canvas, '')
        if finished(State):
            IPython.display.clear_output(wait=True)
            final_msg(State)
            break

Let's draw the board.

In [ ]:
canvas = create_canvas()
draw(gStart, canvas, f'Current value of game for "X": {v}')

Now its time to play.  In the input window that will pop up later, enter your move in the format "`row,col`"  with no space between row and column.  Both `row` and `col` should be integers from the set `{0,1,2}`.  

In [ ]:
play_game(canvas)